<a href="https://colab.research.google.com/github/DhanviSompalli15/personal-expense-tracker/blob/main/personal_expense_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries if not already installed
!pip install seaborn ipywidgets

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from io import BytesIO
import base64
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output

# Set the style for better visualization
plt.style.use('ggplot')

# Initialize an empty DataFrame to store expenses
expenses = pd.DataFrame(columns=['Date', 'Category', 'Description', 'Amount'])

def add_expense(date, category, description, amount):
    """Add a new expense to the DataFrame."""
    global expenses
    try:
        new_expense = pd.DataFrame({
            'Date': [pd.to_datetime(date)],
            'Category': [category],
            'Description': [description],
            'Amount': [float(amount)]
        })
        expenses = pd.concat([expenses, new_expense], ignore_index=True)
    except ValueError as e:
        print(f"Error: {e}")

def plot_to_base64(fig):
    """Convert matplotlib figure to base64 string for HTML display."""
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_str

def visualize_expenses():
    """Generate visualizations for expense data."""
    if expenses.empty:
        return HTML("<h3>No expenses to visualize.</h3>")

    # Create HTML output
    html_output = "<h2>Expense Visualizations</h2>"

    # 1. Monthly Spending Trend
    monthly_expenses = expenses.groupby(expenses['Date'].dt.to_period('M'))['Amount'].sum()
    fig, ax = plt.subplots(figsize=(10, 6))
    monthly_expenses.plot(kind='line', marker='o', ax=ax)
    ax.set_title('Monthly Spending Trend')
    ax.set_xlabel('Month')
    ax.set_ylabel('Total Expenses ($)')
    plt.xticks(rotation=45)
    html_output += f'<img src="data:image/png;base64,{plot_to_base64(fig)}">'
    plt.close(fig)

    # 2. Category-wise Distribution
    category_expenses = expenses.groupby('Category')['Amount'].sum()
    fig, ax = plt.subplots(figsize=(10, 6))
    category_expenses.plot(kind='pie', ax=ax, autopct='%1.1f%%')
    ax.set_title('Expense Distribution by Category')
    html_output += f'<img src="data:image/png;base64,{plot_to_base64(fig)}">'
    plt.close(fig)

    # 3. Top 5 Expenses
    top_expenses = expenses.nlargest(5, 'Amount')[['Description', 'Amount']]
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x='Amount', y='Description', data=top_expenses, ax=ax)
    ax.set_title('Top 5 Expenses')
    ax.set_xlabel('Amount ($)')
    html_output += f'<img src="data:image/png;base64,{plot_to_base64(fig)}">'
    plt.close(fig)

    return HTML(html_output)

# Create UI widgets
date_input = widgets.DatePicker(description='Date:', value=datetime.now())
category_input = widgets.Dropdown(
    options=['Food', 'Utilities', 'Entertainment', 'Transport', 'Other'],
    description='Category:',
    value='Food'
)
description_input = widgets.Text(description='Description:', placeholder='Enter description')
amount_input = widgets.FloatText(description='Amount:', value=0.0)
add_button = widgets.Button(description='Add Expense', button_style='success')
view_button = widgets.Button(description='View Expenses', button_style='info')
visualize_button = widgets.Button(description='Visualize Data', button_style='primary')
output = widgets.Output()

# Define button click handlers
def on_add_button_clicked(b):
    with output:
        clear_output()
        if not description_input.value or amount_input.value <= 0:
            print("Please provide a valid description and amount.")
            return
        add_expense(date_input.value, category_input.value, description_input.value, amount_input.value)
        print("Expense added successfully!")
        display(expenses)

def on_view_button_clicked(b):
    with output:
        clear_output()
        if expenses.empty:
            print("No expenses recorded.")
        else:
            display(expenses)

def on_visualize_button_clicked(b):
    with output:
        clear_output()
        display(visualize_expenses())

# Assign handlers to buttons
add_button.on_click(on_add_button_clicked)
view_button.on_click(on_view_button_clicked)
visualize_button.on_click(on_visualize_button_clicked)

# Display the UI
display(widgets.VBox([
    date_input,
    category_input,
    description_input,
    amount_input,
    widgets.HBox([add_button, view_button, visualize_button]),
    output
]))

# Add sample expenses for demonstration
add_expense('2025-01-01', 'Food', 'Grocery shopping', 150.00)
add_expense('2025-01-02', 'Utilities', 'Electricity bill', 80.00)
add_expense('2025-01-03', 'Entertainment', 'Movie tickets', 25.00)
add_expense('2025-02-01', 'Food', 'Restaurant', 60.00)
add_expense('2025-02-15', 'Transport', 'Gas', 45.00)

<ipython-input-5-125f27affd5c>:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  expenses = pd.concat([expenses, new_expense], ignore_index=True)
